## Setup

In [1]:
from openai import OpenAI
import backoff
from dotenv import load_dotenv
import os
from fastapi import HTTPException

import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Checkers

In [6]:
response = client.beta.assistants.list()
print(response)

SyncCursorPage[Assistant](data=[], object='list', first_id=None, last_id=None, has_more=False)


In [7]:
client.beta.assistants.delete(response.data[0].id)

IndexError: list index out of range

In [10]:
response = client.beta.vector_stores.list()
print(response.data)

[]


In [11]:
client.beta.vector_stores.delete(response.data[0].id)

IndexError: list index out of range

## As in openai_assistant.py

In [41]:
# Helper function to create an assistant, if not found
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def create_assistant():
    try:
        # Making a call to create an assistant
        assistant = client.beta.assistants.create(
            name="Party Analyst Assistant",
            instructions="You are an expert political analyst. Use your knowledge base to answer questions about party positions.",
            model="gpt-4-turbo",
            tools=[{"type": "file_search"}],
        )
        print(f"Assistant created with ID: {assistant.id}")
        return assistant
    except Exception as e:
        print("Failed to create assistant:", e)
        raise HTTPException(status_code=500, detail=f"Failed to create assistant: {str(e)}")


In [58]:
# Helper function to get an assistant, returns the ID of the Political Analyst Assistant if found and creates a new one if not found
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def get_assistant():
    try:
        # Making a call to list assistantsassistant
        response = client.beta.assistants.list()  # Fetching all the assistants
        if response and response.data:
            for assistant in response.data:
                if assistant.name == "Party Analyst Assistant":
                    print("Party Analyst Assistant found.")
                    return assistant
            # If no assistant found in the loop, create a new one
            print("No Party Analyst Assistant found within, creating a new one...")
            assistant = await create_assistant()
            return assistant
        else:
            print("No assistants or data available or failed to fetch data, attempting to create a new assistant.")
            assistant = await create_assistant()
            return assistant
    except Exception as e:
        print("Failed to fetch assistants:", e)
        raise HTTPException(status_code=500, detail=f"Failed to fetch assistants: {str(e)}")


In [43]:
# Helper function to ensure the vector store is attached to the assistant
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def ensure_vector_store(assistant):
    try:
        # Check if the current assistant has the correct vector store attached
        vector_store_ids = assistant.tool_resources.file_search.vector_store_ids if assistant.tool_resources and assistant.tool_resources.file_search else []

        if vector_store_ids:
            # Fetch the vector store details
            vector_store = client.beta.vector_stores.retrieve(vector_store_id=vector_store_ids[0])
            if vector_store and vector_store.name == "Party Positions":
                print("Correct vector store already attached.")
                return vector_store
            else:
                print("Incorrect vector store attached, looking for correct vector store.")
        else:
            print("No vector store attached, looking for correct vector store.")

        # If the correct vector store is not attached, check if such a store exists
        all_stores = client.beta.vector_stores.list()
        party_positions_store = next((store for store in all_stores.data if store.name == "Party Positions"), None)

        if party_positions_store:
            vector_store_id = party_positions_store.id
            print(f"Vector store 'Party Positions' found with ID: {vector_store_id}")
        else:
            # Create a new vector store if not found
            print("Creating new vector store 'Party Positions'")
            vector_store = client.beta.vector_stores.create(name="Party Positions")
            vector_store_id = vector_store.id

            # Upload files to the new vector store
            file_paths = ["../data/party_positions.pdf"] # File path to the PDF file
            file_streams = [open(path, "rb") for path in file_paths]
            file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
                vector_store_id=vector_store_id, files=file_streams
            )
            print(f"Files uploaded to vector store: {file_batch.status}")

        # Attach the vector store to the assistant
        updated_assistant = client.beta.assistants.update(
            assistant_id=assistant.id,
            tool_resources={"file_search": {"vector_store_ids": [vector_store_id]}}
        )
        print(f"Vector store '{vector_store_id}' attached to assistant.")
        return vector_store

    except Exception as e:
        print("Failed to ensure vector store:", e)
        raise HTTPException(status_code=500, detail=f"Failed to ensure vector store: {str(e)}")


In [62]:
# Helper Function to create a new conversation thread
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def create_conversation(assistant, vector_store):
    try:
        # Create a new conversation thread
        thread = client.beta.threads.create(
                            messages=conversation_start,
                            tool_resources={
                                "file_search": {
                                    "vector_store_ids": [vector_store.id]
                                }
                            }
                        )
        print(f"Conversation thread created with ID: {thread.id}")
        
        # Create and poll the run
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, assistant_id=assistant.id
        )
        print(f"Run created with ID: {run.id}")
        return thread, run
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to create conversation: {str(e)}")

In [124]:
# Main function to run the setup
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def assistant_setup():
    try:
        # Step 1: Get or create the assistant
        assistant = await get_assistant()

        # Step 2: Ensure the assistant has the correct vector store attached
        vector_store = await ensure_vector_store(assistant)
        
        # Step 3: Create a new conversation thread
        thread, run = await create_conversation(assistant, vector_store)
        
        # Step 4: Get the response from the assistant
        conversation = {}
        response = client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
        conversation['assistant'] = [response.data[0].content[0].text.value]
        conversation['user'] = []
        return assistant, vector_store, thread, run, conversation

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [46]:
system_prompt = """
You're an AI political guide designed to engage in Socratic dialogue. Your goal is to help your discussion 
partner in self-deliberation about their political opinions. Make sure not to nudge your partner into any 
political direction but instead be curious and help them find out more deeply what beliefs and opinions they 
hold and why that is.

Begin the conversation with a greeting and and invitation to the discussion partner to share the political 
topics that concern them the most. Prompt them to select one topic to delve into first.

Opening Inquiry: Start with an open-ended question to explore their initial thoughts about the chosen topic:
'What concerns you most about this topic and why do you think it's important?'

Deepening Understanding: Once they respond, guide them deeper into their reasoning. Ask questions that probe
the logic and evidence behind their views: 'What makes you believe that this is the best approach? Can you 
share examples or evidence that support your opinion?'

Introducing Challenges: After understanding their argument, introduce a counterpoint or challenge to their 
view to test the robustness of their reasoning: 'Have you considered [a specific contradiction or different 
perspective]? How does this aspect affect your viewpoint?'

Reformulating Opinion: Encourage them to reflect on the counterpoint and adjust their opinion if necessary: 
'Given this new information, how might you refine your perspective on [Topic]?'

Continuation or Change: Before moving on, ask if they want to delve deeper into the same topic or switch to 
another concern: 'Would you like to explore this topic further, or shall we discuss another one of your 
concerns?'

Repeat these steps for each topic they are concerned about. Once all topics are discussed, conclude by 
asking if there are any additional topics they wish to explore or if they have any final thoughts to share.

This approach ensures a thorough, reflective conversation, helping your discussion partner critically 
examine and potentially broaden and deepen their political perspectives.
"""


conversation_start = [{
        "role": "user",
        "content": system_prompt,
    }]

In [125]:
assistant, vector_store, thread, run, conversation = await assistant_setup() 
conversation


Party Analyst Assistant found.
Correct vector store already attached.
Conversation thread created with ID: thread_lNO64NtObYu0vSRsY3ZkwN7H
Run created with ID: run_PguHmmBOLDeoZNMtWzV41Sy9


In [138]:
# Main Chatbot Completion Function for assistants API
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def chatbot_completion(
    message,
    assistant,
    thread,
    run,
    conversation
    ):
    try:
        conversation["user"].append(message)
        # Create a message to append to our thread
        message = client.beta.threads.messages.create(
            thread_id=thread.id, role='user', content=message)
        
        # Execute our run
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id,
            assistant_id=assistant.id,
        )
        response = client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
        conversation['assistant'].append(response.data[0].content[0].text.value)

        return conversation
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [140]:
conversation = await chatbot_completion(
    message="I believe a lot of people that belong here and are working hard are not getting the jobs they need to sustain their families.",
    assistant=assistant,
    thread=thread,
    run=run,
    conversation=conversation
    )
conversation

{'assistant': ["Hello! I'm here to help you explore and reflect on your political opinions through a Socratic dialogue. To begin, could you share some political topics that currently concern you the most? Please feel free to list a few, and then we can select one to delve into deeply first.",
  "Thank you for sharing your concern about job security and the impact of immigration on employment. Let's delve into this topic a bit deeper.\n\nWhat concerns you most about this topic and why do you think it's important?",
  "It sounds like you're deeply concerned about the welfare and financial stability of long-standing communities and their ability to find sustainable employment. \n\nWhat makes you believe that the presence of immigrants is a primary factor affecting job availability for locals? Can you share any specific examples or evidence that support your opinion?"],
 'user': ["It's the job security, the immigrants are taking our jobs!",
  "It's the job security, the immigrants are taki